In [1]:
import pandas as pd 

In [3]:
df = pd.read_excel("../data/finetuning/MedInfo2019-QA-Medications.xlsx")
df.head()

,Question,Focus (Drug),Question Type,Answer,Section Title,URL
0,how does rivatigmine and otc sleep medicine in...,rivastigmine,Interaction,tell your doctor and pharmacist what prescript...,What special precautions should I follow?,https://medlineplus.gov/druginfo/meds/a602009....
1,how does valium affect the brain,Valium,Action,Diazepam is a benzodiazepine that exerts anxio...,CLINICAL PHARMACOLOGY,https://dailymed.nlm.nih.gov/dailymed/drugInfo...
2,what is morphine,morphine,Information,Morphine is a pain medication of the opiate fa...,NaN,https://en.wikipedia.org/wiki/Morphine
3,what are the milligrams for oxycodone e,oxycodone ER,Dose,… 10 mg … 20 mg … 40 mg … 80 mg ...,HOW SUPPLIED,https://dailymed.nlm.nih.gov/dailymed/drugInfo...
4,81% aspirin contain resin and shellac in it. ?,aspirin 81 mg,Ingredient,Inactive Ingredients Ingredient Name,INGREDIENTS AND APPEARANCE,https://dailymed.nlm.nih.gov/dailymed/drugInfo...


In [7]:
unique_drugs = df["Focus (Drug)"].unique()
print(f"Total unique drugs: {len(unique_drugs)}")
print(unique_drugs)

Total unique drugs: 516
['rivastigmine' 'Valium' 'morphine' 'oxycodone ER' 'aspirin 81 mg'
 'desonide ointment' 'Tylenol' 'Breo inhaler' 'Qvar' 'cyclosporine'
 'Synthroid' 'fluvastatin' 'montelukast' 'ciprofloxacin' 'Lansoprazole'
 'fluorouracil' 'Levetiracetam' 'terazosin' 'meloxicam' 'nitroglycerin'
 'vitamin C' 'pregabalin' 'marijuana' 'neupro and ropinirole' 'prevnar'
 'oxycodone' 'lithium' 'MST' 'metformin' 'pravastatin'
 'metoprolol succinate' 'tramadol' 'denosumab' 'barbiturates'
 'pneumonia vaccine' 'insulin' 'medical marijuana' 'clonazepam' 'Levaquin'
 'vitamin D' 'formaldehyde' 'cyclobenzaprine | benzodiazepine'
 'vitamin D3' 'estrone injection' 'Rapaflo' 'levothyroxine' 'azithromycin'
 'Vyvanse' 'TrOOP' 'Flomax' 'insulin glargine'
 'medicines that raise blood sugar' 'metoprolol tartrate'
 'corticosteroids' 'drugs that reduce cholesterol' 'Gardasil' 'cromolyn'
 'lorazepam' 'thymol' 'zolpidem' 'mercaptopurine' 'colchicine'
 'methylphenidate' 'normal saline' 'Zostavax' 'Ciprode

In [8]:
TARGET_DRUGS = [
    "Aspirin", "Ibuprofen", "Paracetamol", "Acetaminophen",
    "Amoxicillin", "Azithromycin", "Ciprofloxacin",
    "Metformin", "Atorvastatin", "Lisinopril",
    "Omeprazole", "Levothyroxine", "Albuterol",
    "Gabapentin", "Sertraline", "Losartan",
    "Vitamin D", "Vitamin B12", "Vitamin C"
]

exist = 0
not_exist = 0
for drug in TARGET_DRUGS:
    if drug in unique_drugs:
        exist += 1
    else:
        not_exist += 1

print(f"Drugs found: {exist}, Drugs not found: {not_exist}")

Drugs found: 5, Drugs not found: 14
